In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#Global attributes
wiki_url = "https://en.wikipedia.org/wiki/"
api_url = "http://www.7timer.info/doc.php?lang=en#web_interface"
    
def coordinates(city):

    try:

        city = requests.get(wiki_url+city)

        city = BeautifulSoup(city.text, features="lxml")

        for item in city.find_all("span", {"class":"geo"}):
            try:
                temp = item.text
                break
            except:
                print(f"Oops! Unable to find the mentioned {city} co-ordinates!")

        flag = temp.split(";")
        
        return flag[0],flag[1]
        
    except:
        return "City doesn't exist!"


def documentation_ppt_amt(dataframe):

    #url = "http://www.7timer.info/doc.php?lang=en#web_interface"

    weather = requests.get(api_url)

    weather_out = BeautifulSoup(weather.text, features="lxml")

    tables = weather_out.find_all("table")
    
    prcpt_amt = []
    row = []

    for i,entry in enumerate(tables[5].find_all("td")):
        try:
            if i >= 67 and i <= 86:
                row.append(entry.text)
                if i%2 == 0:
                    prcpt_amt.append(row)
                    row = []
        except:
            print("Server Error")
            

    df1 = pd.DataFrame(prcpt_amt, columns = ['prec_amount', 'Ppt_Meaning'])
    df1.replace(to_replace = '\r\n\t', regex=True, value='', inplace=True)
    df1["prec_amount"]=df1["prec_amount"].astype(int)
    temp = pd.merge(dataframe,df1,on="prec_amount",how="left")

    
    return temp